In [42]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [58]:
from proj1_helpers import *
DATA_TRAIN_PATH = 'C:/Users/samsung/ML_Projects/project1/data/train/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Understand and clean the dataset

In [44]:
#Firt let's see the shape of our dataset
print('X shape : ', tX.shape, '||| Y shape : ', y.shape, '||| ids shape : ', ids.shape)

#Let's see what values take y :
print('y takes value in : ',np.unique(y)) #we suppose -1 means that the boson haven't been observed, and 1 mean the opposite

X shape :  (250000, 30) ||| Y shape :  (250000,) ||| ids shape :  (250000,)
y takes value in :  [-1.  1.]


In [4]:
print('the label of the first entry on our data is :' ,y[0])

the label of the first entry on our data is : 1.0


Therefore, by seeing that the first label of our dataset is "s" which stands for signal, **y =1.0 means that there is a signal, and y=-1.0 means background**

Then let's see if the data contains NaN values

In [5]:
print('y has NaN values ?',np.isnan(sum(y)))
print('tX has NaN values ?',np.isnan(sum(sum(tX)))) #tX has 2 dimensions so we sum over the two dimensions
print('ids has NaN values ?', np.isnan(sum(ids)))

print(np.where(tX==-999.0)) #several -999.0 values

y has NaN values ? False
tX has NaN values ? False
ids has NaN values ? False
(array([     1,      1,      1, ..., 249999, 249999, 249999], dtype=int64), array([ 4,  5,  6, ..., 26, 27, 28], dtype=int64))


<ipython-input-5-ef7edc514d89>:3: RuntimeWarning: overflow encountered in long_scalars
  print('ids has NaN values ?', np.isnan(sum(ids)))


As decribed on AICrowd, features prefixed with DER are derived from features prefixed with PRI so we can only keep the "raw" features, those prefixed with PRI, without losing information. This will also reduce to size of the data

Plus there is a lot of "-999.0" in our dataset so we must remove them. A firts idea is to replace them by 0. An other idea is to replace these values by the mean or the median. 

In [59]:
from manipulate_data import *

tX = clean_data(tX) 

In [60]:
print('the new shape is :',tX.shape)
np.where(tX==-999.0) #the array is empty : there is no more -999.0 values

the new shape is : (250000, 30)


(array([], dtype=int64), array([], dtype=int64))

# Algorithms :

**1.Least squares with Gradient Descent**

In [63]:
ttt= np.array([[1,2,3], [3,4,5]])
print(ttt)
print(build_poly(ttt,4))


[[1 2 3]
 [3 4 5]]
[[  1.   1.   1.   1.   2.   3.   1.   4.   9.   1.   8.  27.   1.  16.
   81.]
 [  1.   1.   1.   3.   4.   5.   9.  16.  25.  27.  64. 125.  81. 256.
  625.]]


In [64]:
from implementations import *
from manipulate_data import *


#Least Squares - GD
max_iters = 20 
gammas = [0.7, 0.1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX.shape[1],)) #number of parameters = number of features of x

print('Without any polynomial augmentation : degree fixed to 1, gamma varies')
for gamma in gammas:
    w_lsGD, loss_lsGD= least_squares_GD(y, tX, initial_w, max_iters, gamma)
    print('for gamma = {}, loss = {}'.format(gamma,loss_lsGD))


#We do it with polynomial features
degrees=[1, 3, 5, 7, 9, 11]
gamma = 1e-15
print('\nGamma fixed : {}, degree varies'.format(gamma))

for degree in degrees:
    tX_exp = build_poly(tX, degree)
    initial_w = np.zeros(((degree+1)*tX.shape[1],))
    w_lsGD, loss_lsGD= least_squares_GD(y, tX_exp, initial_w, max_iters, gamma)
    print('for degree = {}, loss = {}'.format(degree,loss_lsGD))


Without any polynomial augmentation : degree fixed to 1, gamma varies
for gamma = 0.7, loss = 1.175921591050787e+202
for gamma = 0.1, loss = 1.8434979074371061e+168
for gamma = 0.001, loss = 1.4837295600462847e+88
for gamma = 0.0001, loss = 1.9492650062973896e+47
for gamma = 1e-05, loss = 0.4008444611570252
for gamma = 1e-06, loss = 0.44832327176287684
for gamma = 1e-07, loss = 0.4884308641506899
for gamma = 1e-08, loss = 0.49866753660288426

Gamma fixed : 1e-15, degree varies
for degree = 1, loss = 0.4999999998643966
for degree = 3, loss = 2.1154128119337214e+118
for degree = 5, loss = nan


KeyboardInterrupt: 

In [69]:
#same experiment with tX normalized :
tX_norm, mean_tx, std_tx = standardise(tX)

max_iters = 50 
gammas = [0.7, 0.1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX_norm.shape[1],)) #number of parameters = number of features of x

print('Without any polynomial augmentation : degree fixed to 1, gamma varies')
'''
for gamma in gammas:
    w_lsGD, loss_lsGD= least_squares_GD(y, tX_norm, initial_w, max_iters, gamma)
    print('for gamma = {}, loss = {}'.format(gamma,loss_lsGD))'''


    
#We do it with polynomial features
degrees=[1, 3, 5, 7, 9, 11]
gamma = 1e-10
print('\nGamma fixed : {}, degree varies'.format(gamma))

for degree in degrees:
    tX_exp = build_poly(tX_norm, degree)
    initial_w = np.zeros(((degree+1)*tX_norm.shape[1],))
    w_lsGD, loss_lsGD= least_squares_GD(y, tX_exp, initial_w, max_iters, gamma)
    print('for degree = {}, loss = {}'.format(degree,loss_lsGD))


Without any polynomial augmentation : degree fixed to 1, gamma varies

Gamma fixed : 1e-10, degree varies
for degree = 1, loss = 0.4999999821596929
for degree = 3, loss = 0.49999949287654283
for degree = 5, loss = inf


C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:79: RuntimeWarning: invalid value encountered in subtract
  loss = compute_loss(y, tx, w, lambda_)


for degree = 7, loss = nan


KeyboardInterrupt: 

In [67]:
#Fix the parameters to have a final value and do prediction
max_iters=200
gamma=1e-5
initial_w = np.zeros((tX_norm.shape[1],))
w_lsGD, loss_lsGD= least_squares_GD(y, tX_norm, initial_w, max_iters, gamma)


In [68]:
print(loss_lsGD)

0.4988176737354244


**2.Least squares with SGD**

In [66]:
#Least squares with SGD on normalized data
tX_norm, mean_tx, std_tx = standardise(tX)

max_iters = 20 
gammas = [0.7, 0.1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX_norm.shape[1],)) #number of parameters = number of features of x

print('Without any polynomial augmentation : degree fixed to 1, gamma varies')
for gamma in gammas:
    w_lsSGD, loss_lsSGD= least_squares_SGD(y, tX_norm, initial_w, max_iters, gamma)
    print('for gamma = {}, loss = {}'.format(gamma,loss_lsSGD))

    
#We do it with polynomial features
degrees=[1, 3, 5, 7, 9, 11]
gamma = 1e-10
print('\nGamma fixed : {}, degree varies'.format(gamma))

for degree in degrees:
    tX_exp = build_poly(tX_norm, degree)
    initial_w = np.zeros(((degree+1)*tX_norm.shape[1],))
    w_lsSGD, loss_lsSGD= least_squares_SGD(y, tX_exp, initial_w, max_iters, gamma)
    print('for degree = {}, loss = {}'.format(degree,loss_lsSGD))

Without any polynomial augmentation : degree fixed to 1, gamma varies
for gamma = 0.7, loss = 5.562278705066963e+18
for gamma = 0.1, loss = 4.798443196321069
for gamma = 0.001, loss = 0.48901147610967105
for gamma = 0.0001, loss = 0.49793485597068954
for gamma = 1e-05, loss = 0.4997719114864903
for gamma = 1e-06, loss = 0.49998822885630745
for gamma = 1e-07, loss = 0.49999956760972925
for gamma = 1e-08, loss = 0.4999998937511005

Gamma fixed : 1e-10, degree varies
for degree = 1, loss = 0.49999999164742137
for degree = 3, loss = 0.49999996866101304
for degree = 5, loss = 0.5068218543105351
for degree = 7, loss = 366316.4245951973
for degree = 9, loss = 4.347690980520219e+19
for degree = 11, loss = 2.088939160245814e+29


**3.Ridge Regression**

In [74]:
lambda_ = 1 #doesn't seem to change anything, we almost sure to be overfitting : loss is very very small
tX_exp = build_poly(tX_norm, 7)

w_rr, loss_rr = ridge_regression(y, tX_exp, lambda_)
print('loss :', loss_rr)
print('weight :', w_rr)

loss : 0.3477943467733908
weight : [-6.53073663e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03 -6.53087895e-03
 -6.53087895e-03 -6.53087895e-03  2.83033873e-02 -4.65351591e-02
  1.33752492e-02  1.13086425e-02 -1.41287772e-03  1.50636370e-03
 -2.24173691e-03  1.30614188e-02 -5.28478255e-03  1.24909309e-02
 -3.24594084e-02  1.19562605e-02  2.21375398e-03  4.07053360e-02
 -2.88597912e-04 -2.99449737e-04  8.42267217e-04  2.19747680e-04
  3.42420697e-04  7.05618928e-04  6.91918162e-04  6.83295747e-03
 -2.17331358e-03  7.16212941e-03 -3.47187624e-04 -1.75762923e-04
  2.11018456e-03  1.28398347e-04 -9.33008509e-05  5.248

**4.Logistic regression**

In [76]:
#Logistic regression (GD) on normalized data without regularization
max_iters = 20 
gammas = [0.7, 0.1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX_norm.shape[1],)) #number of parameters = number of features of x

print('Without any polynomial augmentation : degree fixed to 1, gamma varies')
for gamma in gammas:
    w_lr, loss_lr= logistic_regression(y, tX_norm, initial_w, max_iters, gamma)
    print('for gamma = {}, loss = {}'.format(gamma,loss_lr))

    
#We do it with polynomial features
degrees=[1, 3, 5, 7, 9, 11]
gamma = 1e-10
print('\nGamma fixed : {}, degree varies'.format(gamma))

for degree in degrees:
    tX_exp = build_poly(tX_norm, degree)
    initial_w = np.zeros(((degree+1)*tX_norm.shape[1],))
    w_lr, loss_lr= logistic_regression(y, tX_exp, initial_w, max_iters, gamma)
    print('for degree = {}, loss = {}'.format(degree,loss_lr))

Without any polynomial augmentation : degree fixed to 1, gamma varies


C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:42: RuntimeWarning: overflow encountered in exp
  #loss = (1/2) * ( 2*np.sum(np.log(1 + np.exp(z))) - np.dot(y.T, z) - z)
C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:34: RuntimeWarning: overflow encountered in exp
  return np.exp(t) / (1+np.exp(t))
C:\Users\samsung\ML_Projects\project1\scripts\implementations.py:34: RuntimeWarning: invalid value encountered in true_divide
  return np.exp(t) / (1+np.exp(t))


for gamma = 0.7, loss = [nan nan nan ... nan nan nan]
for gamma = 0.1, loss = [nan nan nan ... nan nan nan]
for gamma = 0.001, loss = [nan nan nan ... nan nan nan]
for gamma = 0.0001, loss = [nan nan nan ... nan nan nan]
for gamma = 1e-05, loss = [190553.00520973 190551.53647459 190553.46601892 ... 190552.23651374
 190550.74752928 190551.9057004 ]
for gamma = 1e-06, loss = [147438.62105186 147438.40825538 147439.43093211 ... 147438.65685885
 147438.30835774 147438.93419434]
for gamma = 1e-07, loss = [162227.93825257 162227.95649106 162228.18169502 ... 162228.00538184
 162227.97692149 162228.10852326]
for gamma = 1e-08, loss = [171537.37795747 171537.38343033 171537.40770947 ... 171537.38870024
 171537.38803516 171537.40263179]

Gamma fixed : 1e-10, degree varies
for degree = 1, loss = [172881.17280499 172881.17286537 172881.17310989 ... 172881.17291848
 172881.17291538 172881.17306308]
for degree = 3, loss = [171897.61132664 171897.61075894 171897.62192752 ... 171897.61036134
 171897.6

In [77]:
#Logistic regression (GD) on normalized data with regularization
max_iters = 20 
gammas = [0.7, 0.1, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8] #by testing the results: this seems to be the optimal value
initial_w = np.zeros((tX_norm.shape[1],)) #number of parameters = number of features of x
lambda_ = 1000

print('Without any polynomial augmentation : degree fixed to 1, gamma varies')
for gamma in gammas:
    w_lr, loss_lr= reg_logistic_regression(y, tX_norm, lambda_, initial_w, max_iters, gamma)
    print('for gamma = {}, loss = {}'.format(gamma,loss_lr))

    
#We do it with polynomial features
degrees=[1, 3, 5, 7, 9, 11]
gamma = 1e-10
print('\nGamma fixed : {}, degree varies'.format(gamma))

for degree in degrees:
    tX_exp = build_poly(tX_norm, degree)
    initial_w = np.zeros(((degree+1)*tX_norm.shape[1],))
    w_lr, loss_lr= reg_logistic_regression(y, tX_exp, lambda_, initial_w, max_iters, gamma)
    print('for degree = {}, loss = {}'.format(degree,loss_lr))

Without any polynomial augmentation : degree fixed to 1, gamma varies
for gamma = 0.7, loss = [nan nan nan ... nan nan nan]
for gamma = 0.1, loss = [nan nan nan ... nan nan nan]
for gamma = 0.001, loss = [nan nan nan ... nan nan nan]
for gamma = 0.0001, loss = [1116326.89719379 1116316.40389759 1116340.96770706 ... 1116322.28186011
 1116310.91208137 1116323.50786989]
for gamma = 1e-05, loss = [197523.12122382 197521.66548529 197523.66191656 ... 197522.33019692
 197520.89209248 197522.0181163 ]
for gamma = 1e-06, loss = [147912.92526239 147912.71723089 147913.72514514 ... 147912.96206826
 147912.61979762 147913.23646928]
for gamma = 1e-07, loss = [162264.18093992 162264.19916144 162264.42394742 ... 162264.24796144
 162264.21956528 162264.35092499]
for gamma = 1e-08, loss = [171538.04660189 171538.05207374 171538.07634827 ... 171538.05734265
 171538.05667771 171538.07127157]

Gamma fixed : 1e-10, degree varies
for degree = 1, loss = [172881.1737973  172881.17385768 172881.1741022  ... 17

In [37]:
#Fixed paramters to compute optimal weight for logistic regression
max_iters = 100 
gamma = 1e-06
initial_w = np.zeros((tX_norm.shape[1],))
lambda_ = 0
w_lr, loss_lr= reg_logistic_regression(y, tX_norm, lambda_, initial_w, max_iters, gamma)

## Generate predictions and save ouput in csv format for submission:

In [33]:
DATA_TEST_PATH = 'C:/Users/samsung/ML_Projects/project1/data/test/test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

tX_test = clean_data(tX_test) 

In [35]:
_

array([ 1., -1., -1., ...,  1., -1., -1.])

In [38]:
#Loss on test data
print('loss on test data for least squares = ', compute_reg_ls_loss(_, tX_test, w_lsGD, lambda_=0))
print('loss on test data for logistic regression = ', compute_reg_ls_loss(_, tX_test, w_lr, lambda_=0))

loss on test data for least squares =  0.5299287820655979
loss on test data for logistic regression =  3366.901374460843


In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)